In [1]:
import os
import ast
import random
import logging
import numpy as np
import pandas as pd
import networkx as nx

import matplotlib.pyplot as plt

from collections import OrderedDict, Counter
from tqdm import tqdm_notebook as tqdm

import sys
sys.path.append('..')

In [2]:
a = [(6, 0), (6, 1), (1, 2), (0, 3), (1, 4), (0, 5)]

In [3]:
a = [(0,1), (0,2), (1,3), (1,4), (2,3), (2,4)]

In [4]:
b = [(30, 1), (30, 2), (2, 3), (1,3)]

In [5]:
G = nx.DiGraph()

In [6]:
G.add_edges_from(a)

In [7]:
G = nx.read_graphml("C:/Users/harshasivajit/Documents/Starspace/data/oms/cat_hier_graph.graphml")

In [8]:
len(G.nodes())

55257

In [9]:
def get_root(graph):

    in_degree = nx.in_degree_centrality(graph)

    root = []

    min_v = min(in_degree.values())
    for k, v in in_degree.items():
        if v == min_v:
            root.append(k)

    return root

In [10]:
root = get_root(G)

In [11]:
root

['science']

In [12]:
# to-do: similar for string
def BFS(s): 
    # applicable only for directed graphs
    visited = [0] * (len(G)) 
    traversal = []
    new_edges = []
    queue = [] 

    queue.append(s) 
    visited[s]+=1

    while queue: 

        s = queue.pop(0) 
        traversal.append(s)

        for i in G.neighbors(s): 
            queue.append(i) 
            visited[i] +=1
            if visited[i] != 1:
                pass
#                 new_edges.append((s, i+visited[i]+len(G)))
#                 new_edges.append((s, "{}-{}".format(s, i)))
            else:
                new_edges.append((s, i))
            
    return new_edges

In [23]:
G.in_degree()

InDegreeView({'science': 0, 'arts-and-humanities': 1, 'applied-sciences': 1, 'natural-sciences': 1, 'social-sciences': 1, 'mathematics': 1, 'health-sciences': 1, 'archeology': 2, 'astrology': 1, 'philosophy': 1, 'art': 1, 'religion': 1, 'museology': 1, 'culture-heritage': 2, 'cultural-studies': 1, 'agricultural-science': 1, 'computer-science': 1, 'engineering': 1, 'materials-science': 1, 'food-science': 1, 'aquaculture': 1, 'forensic-science': 1, 'chemistry': 1, 'earth-sciences': 1, 'physics': 1, 'biological-sciences': 1, 'space-sciences': 1, 'interdisciplinary-application-studies': 1, 'decision-sciences': 1, 'history': 1, 'linguistics': 1, 'psychology': 1, 'anthropology': 1, 'law': 1, 'sociology': 1, 'business': 1, 'information-science': 1, 'economics': 1, 'education': 1, 'political-science': 1, 'human-geography': 2, 'geography': 1, 'research-methodology-social-sciences': 1, 'methodological-approaches-social-sciences': 1, 'geometry': 1, 'probability-theory': 1, 'algebra': 1, 'number-t

In [13]:
# Converts DAG to tree. A LOT OF INFORMATION IS LOST THOUGH
def BFS(s): 
    # applicable only for directed graphs
    visited = Counter() 
    traversal = []
    new_edges = []
    queue = [] 
    
    curr_node = s[0]
    queue.append(curr_node) 
    visited[curr_node]+=1
    while queue: 
        
        s = queue.pop(0) 
        traversal.append(s)

        for i in G.neighbors(s): 
            queue.append(i) 
            visited[i] +=1
            if visited[i] != 1:
                pass
#                 new_edges.append((s, i+visited[i]+len(G)))
#                 new_edges.append((s, "{}-{}".format(s, i)))
            else:
                new_edges.append((s, i))

    return new_edges

In [14]:
new_edges = BFS(root)

In [15]:
r = nx.DiGraph()

In [16]:
r.add_edges_from(new_edges)

In [19]:
len(r.nodes())

55257

In [22]:
nx.is_tree(r)

True